## Checks that the event codes are consistent with conditions

This script cross-checks for consistency in `_events_temp3.tsv` after
all corrections have been made notebook.

### Checking for forbidden codes
       Codes 1 and 2 can appear anywhere
       Codes 3 through 6 should appear only in the focus condition.
       Codes 7 through 14 should appear only in the shift condition.
       Codes 199, 201, 202, and 255 are not related to condition.

In [1]:
from hed.tools import BidsTsvDictionary, HedLogger
from hed.util import get_file_list, get_new_dataframe

# Set up the directories and make a dictionary for _events_temp3.tsv
bids_root_path = 'G:\AttentionShift\AttentionShiftWorking'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'run')
skip_cols = ['onset', 'duration', 'sample']

bids_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events_temp3",
                           exclude_dirs=exclude_dirs)
bids_dict = BidsTsvDictionary(bids_files, entities=entities)

In [2]:
status = HedLogger()
print("Isolating the bad codes:")
for key, file, rowcount, columns in bids_dict.iter_tsv_info():
    df_bids = get_new_dataframe(file.file_path)

    focus_cond_mask = df_bids['cond_code'].map(str).isin(['1', '2'])
    shift_cond_mask = df_bids['cond_code'].map(str).isin(['3'])
    focus_event_mask = df_bids['event_code'].map(str).isin(['3', '4', '5', '6'])
    shift_event_mask = df_bids['event_code'].map(str).isin(['7', '8', '9', '10', '11', '12', '13', '14'])
    bad_focus = sum(focus_cond_mask & shift_event_mask)
    if bad_focus:
        indices = df_bids.index[focus_cond_mask & shift_event_mask]
        status.add(key, f"{key} has {bad_focus} shift event codes in a focus condition")
        status.add(key, f"{str(list(indices))}")
    bad_shift = sum(shift_cond_mask & focus_event_mask)
    if bad_shift:
        indices = df_bids.index[shift_cond_mask & focus_event_mask]
        status.add(key, f"{key} has {bad_shift} focus event codes in a shift condition")
        status.add(key, f"{str(list(indices))}")
    bad_cond_mask = df_bids['cond_code'].map(str).isin(['0'])
    if sum(bad_cond_mask):
        status.add(key, f"{key} has {sum(bad_cond_mask)} cond_code values of 0")
        status.add(key, f"{str(list(df_bids.index[bad_cond_mask]))}")
    unknown_mask = sum(df_bids['event_code'].map(str).isin(['255']))
    if sum(unknown_mask):
        status.add(key, f"{key} has {sum(unknown_mask)} event_code values of 255")
        status.add(key, f"{str(list(df_bids.index[unknown_mask]))}")

Isolating the bad codes:


In [3]:
status.print_log()
status.save_log(bids_root_path, sub_path='code', log_name='as_hed_06_final_consistency_check_log.json')


sub-005_run-01
	sub-005_run-01 has 6 event_code values of 255
